In [1]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

pd.set_option('display.max_colwidth', -1)

In [2]:
base_url = 'https://www.cs.umd.edu'

In [3]:
home_url = base_url + '/cscareerfair/students'

In [4]:
response = requests.get(home_url)

home_soup = BeautifulSoup(response.content, 'html.parser')

#soup

In [5]:
# Helper generator function to yield company details link

def get_company_links():
    for c, i in enumerate(home_soup.find_all('div', attrs = {'class': 'pane-content'})):
        if c == 3: # 3 has the info about all attendees
            j = i.find('div', attrs = {'class' : 'view-content'})
            for c2, company in enumerate(j.find_all('div')):
                if c2%3 == 0:
                    yield(company.find('a').get('href'))

In [6]:
# Get links of all companies one at a time and visit them one by one
# Define a generator function that will populate the dataframe

def get_company_vals(prev_generator):
    for link in prev_generator:
        #print(link)
        company_url = base_url + link

        response = requests.get(company_url)
        company_soup = BeautifulSoup(response.content, 'html.parser')

        company_name = company_soup.find('h1', attrs = {'class': 'page-header'}).text

        #print(company_name.text)

        group_header = company_soup.find('div', attrs = {'class': 'group-header'})

    #     print(group_header.find('div', 
    #     attrs = {'class': 'field field-name-field-booth-location field-type-text field-label-inline clearfix'}).find('div', 
    #     attrs = {'class': 'field-item even'}).text)

        location = group_header.find('div', 
        attrs = {'class': 'field field-name-field-booth-location field-type-text field-label-inline clearfix'}).find('div', 
        attrs = {'class': 'field-item even'}).text

        booth_number = group_header.find('div', 
        attrs = {'class': 'field field-name-field-booth-number field-type-text field-label-inline clearfix'}).find('div', 
        attrs = {'class': 'field-item even'}).text

        #print(booth_number)

        group_left = company_soup.find('div', attrs = {'class': 'group-left'})

        positions_available = [group_left.find('div', 
        attrs = {'class': 'field field-name-field-positions-available field-type-text field-label-inline clearfix'}).find('div',
        attrs = {'class': 'field-item even'}).text]

        #print(positions_available)

        opportunities = [group_left.find('div', 
        attrs = {'class': 'field field-name-field-opportunities field-type-text field-label-inline clearfix'}).find('div',
        attrs = {'class': 'field-item even'}).text]

        #print(opportunities)

        group_right = company_soup.find('div', attrs = {'class': 'group-right'})

        citizenship = group_right.find('div', 
        attrs = {'class': 'field field-name-field-citzenship-req field-type-list-text field-label-inline clearfix'}).find('div', 
        attrs = {'class': 'field-item even'}).text

        visa = group_right.find('div', 
        attrs = {'class': 'field field-name-field-visa-sponsor field-type-list-text field-label-inline clearfix'}).find('div', 
        attrs = {'class': 'field-item even'}).text

        #print(company_name, citizenship, visa)

        col_vals = [company_name, citizenship, visa, positions_available, opportunities, location, booth_number]
        
        yield col_vals

    

In [15]:
# Create an empty dataframe with column headers

cols = ['Company', 'Citizenship Required', 'Visa', 'Positions', 'Opportunities', 'Location', 'Booth Number']

data = pd.concat([pd.DataFrame([i], columns = cols) for i in get_company_vals(get_company_links())], ignore_index = True)

data.head()

,Company,Citizenship Required,Visa,Positions,Opportunities,Location,Booth Number
0,3M,no,no,"[Software Engineer, Java Engineer,\nSecurity Engineer, and Software Engineer Internships]","[Full-Time (Entry Level), Internship]",Ballroom,040
1,Accenture,yes,no,"[Software Engineering Analyst, Analytics Analyst, Summer Analyst (Intern)]","[Full-Time (Entry Level), Internship]",Ballroom,115
2,Adobe Systems Inc,no,yes,"[Machine Learning, Software Engineering, Technical PM, Design, Data Science.]","[Full-Time (Entry Level), Internship]",Ballroom,038
3,"Advanced Simulation Technology, Inc. (ASTI)",yes,no,[Web Developer\nSoftware Engineer\nApplication Engineer\nSoftware Engineer Intern],"[Full-Time (Entry Level), Internship]",Ballroom,052
4,Advanced Technology & Research Corporation (ATR),yes,no,"[Computer Scientist, Jr Tech Editor, Jr Engineer, Software Engineer, Mechanical Engineer, Electrical Engineer, FPGA Engineer, JR. Engineer FEM/CAD, Rocket Propulsion Engineer]","[Full-Time (Entry Level), Full-Time (Experienced)]",Ballroom,019


In [16]:
data.shape

(152, 7)

In [17]:
# Keep only non citizenship requirement

no_citizen_data = data[data['Citizenship Required'] == 'no']
no_citizen_data.head()

,Company,Citizenship Required,Visa,Positions,Opportunities,Location,Booth Number
0,3M,no,no,"[Software Engineer, Java Engineer,\nSecurity Engineer, and Software Engineer Internships]","[Full-Time (Entry Level), Internship]",Ballroom,040
2,Adobe Systems Inc,no,yes,"[Machine Learning, Software Engineering, Technical PM, Design, Data Science.]","[Full-Time (Entry Level), Internship]",Ballroom,038
6,Alibaba Group,no,yes,[Software Development Engineer (Full-time & Intern)],"[Full-Time (Entry Level), Full-Time (Experienced), Internship]",Ballroom,008
8,Amazon,no,yes,[Software Development Engineer Internship and Full-Time],"[Full-Time (Entry Level), Internship]",Ballroom,013
10,Amtrak,no,no,[All majors],"[Full-Time (Entry Level), Full-Time (Experienced), Part-Time, Internship]",Foyer A,144


In [19]:
def filter_positions(series):
    lowercase = series['Positions'][0].lower()
    return 'analyst' in lowercase or 'data' in lowercase or 'machine' in lowercase or 'intelligence' in lowercase or 'product' in lowercase

no_citizen_data['Worth it'] = no_citizen_data[['Positions']].apply(filter_positions, axis = 1)

/Users/akshatpant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [20]:
no_citizen_data.head()

,Company,Citizenship Required,Visa,Positions,Opportunities,Location,Booth Number,Worth it
0,3M,no,no,"[Software Engineer, Java Engineer,\nSecurity Engineer, and Software Engineer Internships]","[Full-Time (Entry Level), Internship]",Ballroom,040,False
2,Adobe Systems Inc,no,yes,"[Machine Learning, Software Engineering, Technical PM, Design, Data Science.]","[Full-Time (Entry Level), Internship]",Ballroom,038,True
6,Alibaba Group,no,yes,[Software Development Engineer (Full-time & Intern)],"[Full-Time (Entry Level), Full-Time (Experienced), Internship]",Ballroom,008,False
8,Amazon,no,yes,[Software Development Engineer Internship and Full-Time],"[Full-Time (Entry Level), Internship]",Ballroom,013,False
10,Amtrak,no,no,[All majors],"[Full-Time (Entry Level), Full-Time (Experienced), Part-Time, Internship]",Foyer A,144,False


In [21]:
no_citizen_data[no_citizen_data['Worth it'] == True].shape

(23, 8)

In [22]:
no_citizen_data[no_citizen_data['Worth it'] == True]

,Company,Citizenship Required,Visa,Positions,Opportunities,Location,Booth Number,Worth it
2,Adobe Systems Inc,no,yes,"[Machine Learning, Software Engineering, Technical PM, Design, Data Science.]","[Full-Time (Entry Level), Internship]",Ballroom,038,True
12,Appian,no,yes,[Quality Engineer (+Internship)\r\nSoftware Engineer (+Internship)\r\nProduct Manager\r\nUX Engineer (+Internship)\r\nAssociate Consultant\r\nAssociate Solutions Consultant\r\nAssociate Solutions Engineer\r\nCloud Software Engineer\r\nIT Engineer\r\nSecurity Engineer],"[Full-Time (Entry Level), Internship]",Ballroom,102,True
14,ATPCO,no,yes,"[computer engineer, data analyst, business analyst, commercial strategist]","[Full-Time (Entry Level), Full-Time (Experienced), Part-Time, Internship]",Foyer A,141,True
16,Bank of America,no,yes,[Global Technology Analyst Program\nGlobal Quantitative Analytics Program],"[Full-Time (Entry Level), Internship]",Ballroom,022,True
18,Blackstone,no,yes,[Blackstone Innovations - 2020 Summer Analyst],[Internship],Ballroom,060,True
21,"Brivo Systems, LLC",no,yes,"[Software Engineers, Product Managers, Mobile Engineers, QA Analyst]","[Full-Time (Entry Level), Full-Time (Experienced), Internship]",Ballroom,095,True
27,Charles River Associates,no,yes,[Analyst/Associate (Cyber)],"[Full-Time (Entry Level), Internship]",Ballroom,042,True
28,Chewy.com,no,yes,[Software Engineer I\nSoftware Engineering Intern\nData Engineering Intern],"[Full-Time (Entry Level), Internship]",Foyer A,151,True
39,DisputeSoft,no,yes,"[Software Development Intern (Fall 2019, Spring 2020, and/or Summer 2020)\nIT Analyst (full-time entry-level)]","[Full-Time (Entry Level), Internship]",Ballroom,025,True
40,EAB,no,yes,[Associate Software Engineer\nAssociate Data Engineer],"[Full-Time (Entry Level), Full-Time (Experienced)]",Ballroom,041,True


In [25]:
no_citizen_data.to_csv('..../no_citizen_data.csv', index= False)